<a href="https://colab.research.google.com/github/JayThibs/Machine-Learning-With-Tabular-Data/blob/master/Rossman_Store_Sales_Tabular_Data.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Rossmann Store Sales

## Description

Rossmann operates over 3,000 drug stores in 7 European countries. Currently, Rossmann store managers are tasked with predicting their daily sales for up to six weeks in advance. Store sales are influenced by many factors, including promotions, competition, school and state holidays, seasonality, and locality. With thousands of individual managers predicting sales based on their unique circumstances, the accuracy of results can be quite varied.

## Problem Statement

Create a forecasting model to predict the number of sales in Rossmann stores.

In [0]:
%reload_ext autoreload
%autoreload 2

In [0]:
from fastai import *
from fastai.tabular import *
from fastai.basics import *
import os # for reading from the file

## Data Preparation / Feature Engineering

We will be using [external data](http://files.fast.ai/part2/lesson14/rossmann.tgz) along with the data provided in the Kaggle competition.

In [3]:
!wget -P /content/ http://files.fast.ai/part2/lesson14/rossmann.tgz

--2019-10-08 23:04:25--  http://files.fast.ai/part2/lesson14/rossmann.tgz
Resolving files.fast.ai (files.fast.ai)... 67.205.15.147
Connecting to files.fast.ai (files.fast.ai)|67.205.15.147|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 7730448 (7.4M) [application/x-gtar-compressed]
Saving to: ‘/content/rossmann.tgz’

rossmann.tgz        100%[===================>]   7.37M  --.-KB/s    in 0.1s    

2019-10-08 23:04:25 (52.6 MB/s) - ‘/content/rossmann.tgz’ saved [7730448/7730448]



In [0]:
!tar -xf /content/rossmann.tgz

In [0]:
!rm -rf /content/rossmann.tgz

In [6]:
table_names = ['train', 'store', 'store_states', 'state_names', 'googletrend',
               'weather', 'test']

tables = [pd.read_csv(f'{fname}.csv', low_memory=False) \
          for fname in table_names]
train, store, store_states, state_names, googletrend, weather, test = tables
len(train), len(test) # number of train and test examples

(1017209, 41088)

Let's have a look at what our training set looks like:

In [7]:
train.head()

,Store,DayOfWeek,Date,Sales,Customers,Open,Promo,StateHoliday,SchoolHoliday
0,1,5,2015-07-31,5263,555,1,1,0,1
1,2,5,2015-07-31,6064,625,1,1,0,1
2,3,5,2015-07-31,8314,821,1,1,0,1
3,4,5,2015-07-31,13995,1498,1,1,0,1
4,5,5,2015-07-31,4822,559,1,1,0,1


In [8]:
train.tail()

,Store,DayOfWeek,Date,Sales,Customers,Open,Promo,StateHoliday,SchoolHoliday
1017204,1111,2,2013-01-01,0,0,0,0,a,1
1017205,1112,2,2013-01-01,0,0,0,0,a,1
1017206,1113,2,2013-01-01,0,0,0,0,a,1
1017207,1114,2,2013-01-01,0,0,0,0,a,1
1017208,1115,2,2013-01-01,0,0,0,0,a,1


If we look at `StateHoliday`, we have 0 and a as values. There are actually four types of values: a = public holiday, b = Easter holiday, c = Christmas, and 0 = None. We'll be turning it into a boolean in order to make it more convenient for modeling.

In [0]:
train.StateHoliday = train.StateHoliday != '0'
test.StateHoliday = test.StateHoliday != '0'

In [10]:
train.head(1)

,Store,DayOfWeek,Date,Sales,Customers,Open,Promo,StateHoliday,SchoolHoliday
0,1,5,2015-07-31,5263,555,1,1,False,1


In [11]:
train.tail(1)

,Store,DayOfWeek,Date,Sales,Customers,Open,Promo,StateHoliday,SchoolHoliday
1017208,1115,2,2013-01-01,0,0,0,0,True,1


Since we have a DataFrames, we're going to join them together. We can start by creating a function called `join_df` that will join tables on specific fields:

In [0]:
# The suffix argument '_y' appends all the duplicate field names.
# We will drop these duplicate columns later.
def join_df(left, right, left_on, right_on=None, suffix='_y'):
  if right_on is None: right_on = left_on
  return left.merge(right, how='left', left_on=left_on, right_on=right_on,
                    suffixes=("", suffix))

Let's join the weather/state names.

In [0]:
weather = join_df(weather, state_names, "file", "StateName")

In [14]:
weather.head()

,file,Date,Max_TemperatureC,Mean_TemperatureC,Min_TemperatureC,Dew_PointC,MeanDew_PointC,Min_DewpointC,Max_Humidity,Mean_Humidity,Min_Humidity,Max_Sea_Level_PressurehPa,Mean_Sea_Level_PressurehPa,Min_Sea_Level_PressurehPa,Max_VisibilityKm,Mean_VisibilityKm,Min_VisibilitykM,Max_Wind_SpeedKm_h,Mean_Wind_SpeedKm_h,Max_Gust_SpeedKm_h,Precipitationmm,CloudCover,Events,WindDirDegrees,StateName,State
0,NordrheinWestfalen,2013-01-01,8,4,2,7,5,1,94,87,65,1013,1005,1001,31.0,12.0,4.0,39,26,58.0,5.08,6.0,Rain,215,NordrheinWestfalen,NW
1,NordrheinWestfalen,2013-01-02,7,4,1,5,3,2,93,85,78,1028,1022,1014,31.0,14.0,10.0,24,16,NaN,0.00,6.0,Rain,225,NordrheinWestfalen,NW
2,NordrheinWestfalen,2013-01-03,11,8,6,10,8,4,100,93,77,1035,1030,1026,31.0,8.0,2.0,26,21,NaN,1.02,7.0,Rain,240,NordrheinWestfalen,NW
3,NordrheinWestfalen,2013-01-04,9,9,8,9,9,8,100,94,87,1036,1035,1034,11.0,5.0,2.0,23,14,NaN,0.25,7.0,Rain,263,NordrheinWestfalen,NW
4,NordrheinWestfalen,2013-01-05,8,8,7,8,7,6,100,94,87,1035,1034,1033,10.0,6.0,3.0,16,10,NaN,0.00,7.0,Rain,268,NordrheinWestfalen,NW


To capture the trend and cyclical behavior as a function of time, we will be using `googletrend` and add new columns based on the week column so that we have extra granularity.

We'll also change the statename 'NI' with 'HB,NI' to make it match the rest of our data.

In [15]:
googletrend.head()

,file,week,trend
0,Rossmann_DE_SN,2012-12-02 - 2012-12-08,96
1,Rossmann_DE_SN,2012-12-09 - 2012-12-15,95
2,Rossmann_DE_SN,2012-12-16 - 2012-12-22,91
3,Rossmann_DE_SN,2012-12-23 - 2012-12-29,48
4,Rossmann_DE_SN,2012-12-30 - 2013-01-05,67


In [0]:
googletrend['Date'] = googletrend.week.str.split(' - ', expand=True)[0]
googletrend['State'] = googletrend.file.str.split('_', expand=True)[2]
googletrend.loc[googletrend.State=='NI', "State"] = 'HB,NI'

In [17]:
googletrend.head()

,file,week,trend,Date,State
0,Rossmann_DE_SN,2012-12-02 - 2012-12-08,96,2012-12-02,SN
1,Rossmann_DE_SN,2012-12-09 - 2012-12-15,95,2012-12-09,SN
2,Rossmann_DE_SN,2012-12-16 - 2012-12-22,91,2012-12-16,SN
3,Rossmann_DE_SN,2012-12-23 - 2012-12-29,48,2012-12-23,SN
4,Rossmann_DE_SN,2012-12-30 - 2013-01-05,67,2012-12-30,SN


In [0]:
def add_datepart(df, fldname, drop=True, time=False):
    "Helper function that adds columns relevant to a date."
    fld = df[fldname]
    fld_dtype = fld.dtype
    if isinstance(fld_dtype, pd.core.dtypes.dtypes.DatetimeTZDtype):
        fld_dtype = np.datetime64

    if not np.issubdtype(fld_dtype, np.datetime64):
        df[fldname] = fld = pd.to_datetime(fld, infer_datetime_format=True)
    targ_pre = re.sub('[Dd]ate$', '', fldname)
    attr = ['Year', 'Month', 'Week', 'Day', 'Dayofweek', 'Dayofyear',
            'Is_month_end', 'Is_month_start', 'Is_quarter_end', 'Is_quarter_start', 'Is_year_end', 'Is_year_start']
    if time: attr = attr + ['Hour', 'Minute', 'Second']
    for n in attr: df[targ_pre + n] = getattr(fld.dt, n.lower())
    df[targ_pre + 'Elapsed'] = fld.astype(np.int64) // 10 ** 9
    if drop: df.drop(fldname, axis=1, inplace=True)

In [0]:
add_datepart(weather, "Date", drop=False)
add_datepart(googletrend, "Date", drop=False)
add_datepart(train, "Date", drop=False)
add_datepart(test, "Date", drop=False)

The `googletrend` data has a special category for the whole of Germany, so we'll filter everything else out.

In [0]:
trend_de = googletrend[googletrend.file == 'Rossmann_DE']

We can now outer join all of our data into a single dataframe.

Outer Join is often easier (better) than inner join. We just need to check if there are any null values after we've joined the tables. This avoids any mistakes we might make. For example, if some of the data between tables is not the same, we will get null values. It's worth it to check this before moving forward.

In [0]:
store = join_df(store, store_states, "Store")

In [22]:
len(store[store.State.isnull()])

0

No null values. Now let's join the rest of our dataframes.

In [23]:
joined = join_df(train, store, "Store")
joined_test = join_df(test, store, "Store")
len(joined[joined.StoreType.isnull()]),len(joined_test[joined_test.StoreType.isnull()])

(0, 0)

In [24]:
joined = join_df(joined, googletrend, ["State","Year", "Week"])
joined_test = join_df(joined_test, googletrend, ["State","Year", "Week"])
len(joined[joined.trend.isnull()]),len(joined_test[joined_test.trend.isnull()])

(0, 0)

In [25]:
joined = joined.merge(trend_de, 'left', ["Year", "Week"], suffixes=('', '_DE'))
joined_test = joined_test.merge(trend_de, 'left', ["Year", "Week"], suffixes=('', '_DE'))
len(joined[joined.trend_DE.isnull()]),len(joined_test[joined_test.trend_DE.isnull()])

(0, 0)

In [26]:
joined = join_df(joined, weather, ["State","Date"])
joined_test = join_df(joined_test, weather, ["State","Date"])
len(joined[joined.Mean_TemperatureC.isnull()]),len(joined_test[joined_test.Mean_TemperatureC.isnull()])

(0, 0)

In [27]:
joined.head()

,Store,DayOfWeek,Date,Sales,Customers,Open,Promo,StateHoliday,SchoolHoliday,Year,Month,Week,Day,Dayofweek,Dayofyear,Is_month_end,Is_month_start,Is_quarter_end,Is_quarter_start,Is_year_end,Is_year_start,Elapsed,StoreType,Assortment,CompetitionDistance,CompetitionOpenSinceMonth,CompetitionOpenSinceYear,Promo2,Promo2SinceWeek,Promo2SinceYear,PromoInterval,State,file,week,trend,Date_y,Month_y,Day_y,Dayofweek_y,Dayofyear_y,...,Is_year_end_DE,Is_year_start_DE,Elapsed_DE,file_y,Max_TemperatureC,Mean_TemperatureC,Min_TemperatureC,Dew_PointC,MeanDew_PointC,Min_DewpointC,Max_Humidity,Mean_Humidity,Min_Humidity,Max_Sea_Level_PressurehPa,Mean_Sea_Level_PressurehPa,Min_Sea_Level_PressurehPa,Max_VisibilityKm,Mean_VisibilityKm,Min_VisibilitykM,Max_Wind_SpeedKm_h,Mean_Wind_SpeedKm_h,Max_Gust_SpeedKm_h,Precipitationmm,CloudCover,Events,WindDirDegrees,StateName,Year_y,Month_y,Week_y,Day_y,Dayofweek_y,Dayofyear_y,Is_month_end_y,Is_month_start_y,Is_quarter_end_y,Is_quarter_start_y,Is_year_end_y,Is_year_start_y,Elapsed_y
0,1,5,2015-07-31,5263,555,1,1,False,1,2015,7,31,31,4,212,True,False,False,False,False,False,1438300800,c,a,1270.0,9.0,2008.0,0,NaN,NaN,NaN,HE,Rossmann_DE_HE,2015-08-02 - 2015-08-08,85,2015-08-02,8,2,6,214,...,False,False,1438473600,Hessen,23,16,8,9,6,3,98,54,18,1021,1018,1015,31.0,15.0,10.0,24,11,NaN,0.0,1.0,Fog,13,Hessen,2015,7,31,31,4,212,True,False,False,False,False,False,1438300800
1,2,5,2015-07-31,6064,625,1,1,False,1,2015,7,31,31,4,212,True,False,False,False,False,False,1438300800,a,a,570.0,11.0,2007.0,1,13.0,2010.0,"Jan,Apr,Jul,Oct",TH,Rossmann_DE_TH,2015-08-02 - 2015-08-08,80,2015-08-02,8,2,6,214,...,False,False,1438473600,Thueringen,19,13,7,9,6,3,100,62,25,1021,1019,1017,10.0,10.0,10.0,14,11,NaN,0.0,4.0,Fog,309,Thueringen,2015,7,31,31,4,212,True,False,False,False,False,False,1438300800
2,3,5,2015-07-31,8314,821,1,1,False,1,2015,7,31,31,4,212,True,False,False,False,False,False,1438300800,a,a,14130.0,12.0,2006.0,1,14.0,2011.0,"Jan,Apr,Jul,Oct",NW,Rossmann_DE_NW,2015-08-02 - 2015-08-08,86,2015-08-02,8,2,6,214,...,False,False,1438473600,NordrheinWestfalen,21,13,6,10,7,4,100,61,24,1022,1019,1017,31.0,14.0,10.0,14,5,NaN,0.0,2.0,Fog,354,NordrheinWestfalen,2015,7,31,31,4,212,True,False,False,False,False,False,1438300800
3,4,5,2015-07-31,13995,1498,1,1,False,1,2015,7,31,31,4,212,True,False,False,False,False,False,1438300800,c,c,620.0,9.0,2009.0,0,NaN,NaN,NaN,BE,Rossmann_DE_BE,2015-08-02 - 2015-08-08,74,2015-08-02,8,2,6,214,...,False,False,1438473600,Berlin,19,14,9,9,7,4,94,61,30,1019,1017,1014,10.0,10.0,10.0,23,16,NaN,0.0,6.0,NaN,282,Berlin,2015,7,31,31,4,212,True,False,False,False,False,False,1438300800
4,5,5,2015-07-31,4822,559,1,1,False,1,2015,7,31,31,4,212,True,False,False,False,False,False,1438300800,a,a,29910.0,4.0,2015.0,0,NaN,NaN,NaN,SN,Rossmann_DE_SN,2015-08-02 - 2015-08-08,82,2015-08-02,8,2,6,214,...,False,False,1438473600,Sachsen,20,15,10,8,6,5,82,55,26,1020,1018,1016,10.0,10.0,10.0,14,11,NaN,0.0,4.0,NaN,290,Sachsen,2015,7,31,31,4,212,True,False,False,False,False,False,1438300800


We can now drop all rows with the _y suffix.

In [0]:
for df in (joined, joined_test):
  for c in df.columns:
    if c.endswith('_y'):
      if c in df.columns: df.drop(c, inplace=True, axis=1)

An issues that comes up often enough is missing values. Many models have problems when missing values are present. Therefore, we will be replacing these missing values with an arbitrary signal value that doesn't other appear in the data. In other words, our missing values come from the date columns so we will give the missing values a date in the year 1900.

In [0]:
for df in (joined, joined_test):
  df['CompetitionOpenSinceYear'] = df.CompetitionOpenSinceYear.fillna(1900).astype(np.int32)
  df['CompetitionOpenSinceMonth'] = df.CompetitionOpenSinceMonth.fillna(1).astype(np.int32)
  df['Promo2SinceYear'] = df.Promo2SinceYear.fillna(1900).astype(np.int32)
  df['Promo2SinceWeek'] = df.Promo2SinceWeek.fillna(1).astype(np.int32)

In [0]:
for df in (joined,joined_test):
    df["CompetitionOpenSince"] = pd.to_datetime(dict(year=df.CompetitionOpenSinceYear, 
                                                     month=df.CompetitionOpenSinceMonth, day=15))
    df["CompetitionDaysOpen"] = df.Date.subtract(df.CompetitionOpenSince).dt.days

In [0]:
for df in (joined,joined_test):
    df.loc[df.CompetitionDaysOpen<0, "CompetitionDaysOpen"] = 0
    df.loc[df.CompetitionOpenSinceYear<1990, "CompetitionDaysOpen"] = 0

In [32]:
for df in (joined,joined_test):
    df["CompetitionMonthsOpen"] = df["CompetitionDaysOpen"]//30
    df.loc[df.CompetitionMonthsOpen>24, "CompetitionMonthsOpen"] = 24
joined.CompetitionMonthsOpen.unique()

array([24,  3, 19,  9,  0, 16, 17,  7, 15, 22, 11, 13,  2, 23, 12,  4, 10,  1, 14, 20,  8, 18,  6, 21,  5])

In [33]:
!pip install isoweek
from isoweek import Week

In [0]:
for df in (joined,joined_test):
    df["Promo2Since"] = pd.to_datetime(df.apply(lambda x: Week(
        x.Promo2SinceYear, x.Promo2SinceWeek).monday(), axis=1))
    df["Promo2Days"] = df.Date.subtract(df["Promo2Since"]).dt.days

In [0]:
for df in (joined,joined_test):
    df.loc[df.Promo2Days<0, "Promo2Days"] = 0
    df.loc[df.Promo2SinceYear<1990, "Promo2Days"] = 0
    df["Promo2Weeks"] = df["Promo2Days"]//7
    df.loc[df.Promo2Weeks<0, "Promo2Weeks"] = 0
    df.loc[df.Promo2Weeks>25, "Promo2Weeks"] = 25
    df.Promo2Weeks.unique()

In [0]:
joined.to_pickle('/content/joined')
joined_test.to_pickle('/content/joined_test')

## Durations

To squeeze a little bit more out of our time series dataset, we can do the following:

*   Running Averages
*   Time until next event
*   Time since last event

This is important because we are giving our model a chance to take into the effect caused by the duration between two events.

For example, in our case, our model may learn that if two sales happen within 10 days of each other, we see less of a profit than if we waited 30 days.

In our to count the time between events, we (obviously) need to take into account the time data. The time data is represented as a function of each new row.

Our function will be used to create a new set of rows that count the number of days that has elapsed between two events. Theoretically, the events could be sales, state holidays, school closures, etc.

We'll define the function `get_elapsed` and give it a particular field `fld` to monitor the time elapsed since the last event. The counter will restart every time a new event comes along.



In [0]:
def get_elapsed(fld, pre):
  day1 = np.timedelta64(1, 'D')
  last_date = np.datetime64()
  last_store = 0
  res = []

  for s, v, d in zip(df.Store.values, df[fld].values, df.Date.values):
    if s != last_store:
      last_date = np.datetime64()
      last_store = s
    if v: last_date = d
    res.append(((d-last_date).astype('timedelta64[D]')/day1))
  df[pre+fld] = res

We'll be applying `get_elapsed` to the following columns:

In [0]:
columns = ["Date", "Store", "Promo", "StateHoliday", "SchoolHoliday"]

In [0]:
df = train[columns].append(test[columns])